# DeepProtT5: Fine-tuned T5 models on DeepProtein Benchmark

## 1. Install required packages

In [1]:
!pip install git+https://github.com/bp-kelley/descriptastorus
!pip install lmdb seaborn wandb pydantic DeepPurpose
!pip install transformers bitsandbytes
!pip install accelerate>=0.26.0
!pip install SentencePiece einops rdchiral peft
!pip install numpy==1.23.5 pandas==1.5.3 scikit-learn==1.2.2
!pip install datasets

  Cloning https://github.com/bp-kelley/descriptastorus to /tmp/pip-req-build-w5gp_n3d
  Running command git clone --filter=blob:none --quiet https://github.com/bp-kelley/descriptastorus /tmp/pip-req-build-w5gp_n3d
  Resolved https://github.com/bp-kelley/descriptastorus to commit 9a190343bcd3cfd35142d378d952613bcac40797
  Preparing metadata (setup.py) ... done
  Using cached numpy-1.23.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.3 kB)
  Using cached pandas-1.5.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
Using cached numpy-1.23.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)
Using cached pandas-1.5.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.3
    Uninsta

In [2]:
!pip install torch==2.3.0 torchvision==0.18.0 torchaudio==2.3.0 --index-url https://download.pytorch.org/whl/cu118
!pip install  dgl -f https://data.dgl.ai/wheels/torch-2.3/cu118/repo.html
!pip install PyTDC

Looking in indexes: https://download.pytorch.org/whl/cu118
Looking in links: https://data.dgl.ai/wheels/torch-2.3/cu118/repo.html
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached pandas-2.2.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
Using cached pandas-2.2.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.1 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency confl

Install DeepProtein lib

In [53]:
!git clone https://github.com/jiaqingxie/DeepProtein.git

fatal: destination path 'DeepProtein' already exists and is not an empty directory.


## 2. Single Protein Regression (Inference) with DeepProtT5

For the single protein regression, each input is a pair of protein and a floating-point number

First we define a T5RegreesionModel class that is used for inference.

In [7]:
from transformers import (
    T5Config,
    T5EncoderModel,
    T5Tokenizer,
    PreTrainedModel,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
)

In [16]:
import torch.nn as nn
class T5RegressionModel(PreTrainedModel):

    config_class = T5Config

    def __init__(self, config, d_model=None):
        super().__init__(config)

        self.encoder = T5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_uniref50")


        hidden_dim = d_model if d_model is not None else config.d_model
        self.regression_head = nn.Linear(hidden_dim, 1)

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        labels=None,
        **kwargs
    ):
        encoder_outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        hidden_states = encoder_outputs.last_hidden_state

        mask = attention_mask.unsqueeze(-1)
        pooled_output = (hidden_states * mask).sum(dim=1) / mask.sum(dim=1)
        logits = self.regression_head(pooled_output).squeeze(-1)  # [batch_size]


        loss = None
        if labels is not None:
            labels = labels.float()
            loss = nn.MSELoss()(logits, labels)

        return {
            "loss": loss,
            "logits": logits
        }

Then we load test from Beta and use DeepProtT5 to do inference on Beta test

In [17]:
import os
import sys
import argparse
import torch
import wandb
from transformers import T5Tokenizer

import sys
sys.path.append("/content/DeepProtein/")
from DeepProtein.load_dataset import *
from DeepProtein.dataset import Beta_lactamase
import DeepProtein.ProteinPred as models

path = "/content/DeepProtein/"
test = Beta_lactamase(path + '/DeepProtein/data', 'test')
test_protein_processed, test_target, test_protein_idx = collate_fn(test)
tokenizer = T5Tokenizer.from_pretrained("jiaxie/DeepProtT5-Beta", do_lower_case=False)
model = T5RegressionModel.from_pretrained("jiaxie/DeepProtT5-Beta", torch_dtype=torch.bfloat16).to("cuda")


model.eval()


predictions = []
true_labels = []

with torch.no_grad():
    for seq, target in zip(test_protein_processed, test_target):
        seq = " ".join(seq)
        encoded_input = tokenizer(seq, return_tensors="pt").to("cuda")
        outputs = model(**encoded_input)

        pred = float(outputs["logits"].cpu().squeeze().to(torch.float32).numpy())
        predictions.append(pred)
        true_labels.append(target.numpy()[0])


from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from scipy.stats import pearsonr, spearmanr

r2 =spearmanr(true_labels, predictions)[0]
mae = mean_absolute_error(true_labels, predictions)
mse = mean_squared_error(true_labels, predictions)


print(f"Spearman R² Score: {r2:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")


config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/11.3G [00:00<?, ?B/s]

Spearman R² Score: 0.8733
Mean Absolute Error (MAE): 0.0837
Mean Squared Error (MSE): 0.0161


In [18]:
import gc
import torch

def clear_memory():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()
clear_memory()


## 3. Single Protein Binary Classification (Inference) with DeepProtT5

For the single protein classification, each input is a pair of protein and a label (int)

First we define a T5ClassificationModel class that is used for inference.

In [23]:
class T5ClassificationModel(PreTrainedModel):
    config_class = T5Config

    def __init__(self, config, d_model=None, num_classes=2):
        super().__init__(config)
        self.num_classes = num_classes

        self.encoder = T5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_uniref50")

        hidden_dim = d_model if d_model is not None else config.d_model
        self.classification_head = nn.Linear(hidden_dim, num_classes)

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        labels=None,
        **kwargs
    ):
        encoder_outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        hidden_states = encoder_outputs.last_hidden_state

        mask = attention_mask.unsqueeze(-1)
        pooled_output = (hidden_states * mask).sum(dim=1) / mask.sum(dim=1)
        logits = self.classification_head(pooled_output)  # [batch_size, num_classes]

        loss = None
        if labels is not None:
            labels = labels.to(torch.bfloat16)
            loss = nn.CrossEntropyLoss()(logits, labels)

        return {
            "loss": loss,
            "logits": logits
        }

Then we load from SubCellular-Binary and do inference with DeepProtT5

In [28]:
import torch
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score, accuracy_score

test = BinarySubcellular(path + '/DeepProtein/data', 'test')
test_protein_processed, test_target, test_protein_idx = collate_fn(test)


from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("jiaxie/DeepProtT5-SubCellular-Binary", do_lower_case=False, force_download=True)
model = T5ClassificationModel.from_pretrained("jiaxie/DeepProtT5-SubCellular-Binary", torch_dtype=torch.bfloat16).to("cuda")

model.eval()

predictions = []
true_labels = []

with torch.no_grad():
    for seq, target in zip(test_protein_processed, test_target):
        seq = " ".join(seq)
        encoded_input = tokenizer(seq, return_tensors="pt").to("cuda")
        outputs = model(**encoded_input)

        logits = outputs["logits"].cpu().to(torch.float32).squeeze()

        if logits.ndim == 0:
            pred = float(logits.numpy())
        else:
            pred = logits.numpy()

        predictions.append(pred)
        true_labels.append(target.numpy()[0])

predictions = torch.tensor(predictions, dtype=torch.float32)

if predictions.ndim == 1:
    probs = torch.sigmoid(predictions).numpy()
    binary_preds = (probs >= 0.5).astype(int)
else:
    probs = torch.softmax(predictions, dim=1).numpy()
    binary_preds = probs.argmax(axis=1)

true_labels = torch.tensor(true_labels).numpy()

accuracy = accuracy_score(true_labels, binary_preds)
f1 = f1_score(true_labels, binary_preds, average="weighted")

if predictions.ndim == 1:
    roc_auc = roc_auc_score(true_labels, probs)
    pr_auc = average_precision_score(true_labels, probs)
    print(f"ROC-AUC Score: {roc_auc:.4f}")
    print(f"PR-AUC Score: {pr_auc:.4f}")

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")


tokenizer_config.json:   0%|          | 0.00/20.4k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/238k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.4k [00:00<?, ?B/s]

Accuracy: 0.9217
F1 Score: 0.9218


## 4. Single Protein Multi Classification (Inference) with DeepProtT5

In this part we test DeepProt5 on Fluoscence

In [29]:
class T5ClassificationModel(PreTrainedModel):
    config_class = T5Config

    def __init__(self, config, d_model=None, num_classes=10):
        super().__init__(config)
        self.num_classes = num_classes

        self.encoder = T5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_uniref50")

        hidden_dim = d_model if d_model is not None else config.d_model
        self.classification_head = nn.Linear(hidden_dim, num_classes)

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        labels=None,
        **kwargs
    ):
        encoder_outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        hidden_states = encoder_outputs.last_hidden_state

        mask = attention_mask.unsqueeze(-1)
        pooled_output = (hidden_states * mask).sum(dim=1) / mask.sum(dim=1)
        logits = self.classification_head(pooled_output)  # [batch_size, num_classes]

        loss = None
        if labels is not None:
            labels = labels.to(torch.bfloat16)
            loss = nn.CrossEntropyLoss()(logits, labels)

        return {
            "loss": loss,
            "logits": logits
        }

In [31]:
import torch
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score, accuracy_score

test = Subcellular(path + '/DeepProtein/data', 'test')
test_protein_processed, test_target, test_protein_idx = collate_fn(test)


from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("jiaxie/DeepProtT5-SubCellular", do_lower_case=False, force_download=True)
model = T5ClassificationModel.from_pretrained("jiaxie/DeepProtT5-SubCellular", torch_dtype=torch.bfloat16).to("cuda")

model.eval()

predictions = []
true_labels = []

with torch.no_grad():
    for seq, target in zip(test_protein_processed, test_target):
        seq = " ".join(seq)
        encoded_input = tokenizer(seq, return_tensors="pt").to("cuda")
        outputs = model(**encoded_input)

        logits = outputs["logits"].cpu().to(torch.float32).squeeze()

        if logits.ndim == 0:
            pred = float(logits.numpy())
        else:
            pred = logits.numpy()

        predictions.append(pred)
        true_labels.append(target.numpy()[0])

predictions = torch.tensor(predictions, dtype=torch.float32)

if predictions.ndim == 1:
    probs = torch.sigmoid(predictions).numpy()
    binary_preds = (probs >= 0.5).astype(int)
else:
    probs = torch.softmax(predictions, dim=1).numpy()
    binary_preds = probs.argmax(axis=1)

true_labels = torch.tensor(true_labels).numpy()

accuracy = accuracy_score(true_labels, binary_preds)
f1 = f1_score(true_labels, binary_preds, average="weighted")

if predictions.ndim == 1:
    roc_auc = roc_auc_score(true_labels, probs)
    pr_auc = average_precision_score(true_labels, probs)
    print(f"ROC-AUC Score: {roc_auc:.4f}")
    print(f"PR-AUC Score: {pr_auc:.4f}")

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")


tokenizer_config.json:   0%|          | 0.00/20.4k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/238k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.4k [00:00<?, ?B/s]

model.safetensors:   1%|          | 21.0M/2.42G [00:00<?, ?B/s]

Accuracy: 0.8269
F1 Score: 0.8252


## 5. Protein Pair Regression (Inference) with DeepProtT5

We do the inference on PPI Affinity dataset

In [36]:
class T5PairRegressionModel(PreTrainedModel):
    config_class = T5Config

    def __init__(self, config, d_model=None):
        super().__init__(config)
        self.encoder = T5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_uniref50")
        hidden_dim = d_model if d_model is not None else config.d_model
        self.regression_head = nn.Linear(hidden_dim, 1)

    def forward(self, input_ids=None, attention_mask=None, labels=None, **kwargs):
        encoder_outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        hidden_states = encoder_outputs.last_hidden_state

        mask = attention_mask.unsqueeze(-1)
        pooled_output = (hidden_states * mask).sum(dim=1) / mask.sum(dim=1)
        logits = self.regression_head(pooled_output).squeeze(-1)  # [batch_size]

        loss = None
        if labels is not None:
            labels = labels.to(torch.bfloat16)
            loss = nn.MSELoss()(logits, labels)

        return {"loss": loss, "logits": logits}

In [39]:
import torch
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score, accuracy_score

test = PPI_Affinity(path + '/DeepProtein/data', 'test')
test_1, test_2, test_target, test_protein_idx = collate_fn_ppi(test)


from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("jiaxie/DeepProtT5-PPI-Affinity", do_lower_case=False, force_download=True)
model = T5PairRegressionModel.from_pretrained("jiaxie/DeepProtT5-PPI-Affinity", torch_dtype=torch.bfloat16).to("cuda")

model.eval()

predictions = []
true_labels = []

with torch.no_grad():
    for seq1, seq2, target in zip(test_1, test_2, test_target):
        seq_1 = " ".join(seq1[:200])
        seq_2 = " ".join(seq2[:200])
        encoded_input = tokenizer(seq_1, seq_2, padding="max_length", truncation=True,max_length=800, return_tensors="pt").to("cuda")
        outputs = model(**encoded_input)

        pred = float(outputs["logits"].cpu().squeeze().to(torch.float32).numpy())
        predictions.append(pred)
        true_labels.append(target.numpy()[0])


from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from scipy.stats import pearsonr, spearmanr

r2 =pearsonr(true_labels, predictions)[0]
mae = mean_absolute_error(true_labels, predictions)
mse = mean_squared_error(true_labels, predictions)


print(f"Pearson R² Score: {r2:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")


tokenizer_config.json:   0%|          | 0.00/20.4k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/238k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.4k [00:00<?, ?B/s]

Pearson R² Score: 0.6431
Mean Absolute Error (MAE): 1.8598
Mean Squared Error (MSE): 5.8185


## 6. Protein Pair Binary Classification (Inference) with DeepProtT5

We do the inference on Human PPI dataset

In [43]:
class T5ClassificationModel(PreTrainedModel):
    config_class = T5Config

    def __init__(self, config, d_model=None, num_classes=2):
        super().__init__(config)
        self.num_classes = num_classes

        self.encoder = T5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_uniref50")

        hidden_dim = d_model if d_model is not None else config.d_model
        self.classification_head = nn.Linear(hidden_dim, num_classes)

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        labels=None,
        **kwargs
    ):
        encoder_outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        hidden_states = encoder_outputs.last_hidden_state

        mask = attention_mask.unsqueeze(-1)
        pooled_output = (hidden_states * mask).sum(dim=1) / mask.sum(dim=1)
        logits = self.classification_head(pooled_output)  # [batch_size, num_classes]

        loss = None
        if labels is not None:
            labels = labels.long()
            loss = nn.CrossEntropyLoss()(logits, labels)

        return {
            "loss": loss,
            "logits": logits
        }


In [47]:
import torch
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score, accuracy_score

test = HUMAN_PPI(path + '/DeepProtein/data', 'test')
test_1, test_2, test_target, test_protein_idx = collate_fn_ppi(test)


from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("jiaxie/DeepProtT5-Human", do_lower_case=False, force_download=True)
model = T5ClassificationModel.from_pretrained("jiaxie/DeepProtT5-Human", torch_dtype=torch.bfloat16).to("cuda")

model.eval()

predictions = []
true_labels = []

with torch.no_grad():
    for seq1, seq2, target in zip(test_1, test_2, test_target):
        seq_1 = " ".join(seq1[:100])
        seq_2 = " ".join(seq2[:100])
        encoded_input = tokenizer(seq_1, seq_2, padding=True, truncation=True,max_length=400, return_tensors="pt").to("cuda")
        outputs = model(**encoded_input)
        logits = outputs["logits"].cpu().to(torch.float32).squeeze()

        if logits.ndim == 0:
            pred = float(logits.numpy())
        else:
            pred = logits.numpy()

        predictions.append(pred)
        true_labels.append(target.numpy()[0])

predictions = torch.tensor(predictions, dtype=torch.float32)

if predictions.ndim == 1:
    probs = torch.sigmoid(predictions).numpy()
    binary_preds = (probs >= 0.5).astype(int)
else:
    probs = torch.softmax(predictions, dim=1).numpy()
    binary_preds = probs.argmax(axis=1)

true_labels = torch.tensor(true_labels).numpy()

accuracy = accuracy_score(true_labels, binary_preds)
f1 = f1_score(true_labels, binary_preds)

if predictions.ndim == 1:
    roc_auc = roc_auc_score(true_labels, probs)
    pr_auc = average_precision_score(true_labels, probs)
    print(f"ROC-AUC Score: {roc_auc:.4f}")
    print(f"PR-AUC Score: {pr_auc:.4f}")

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")

tokenizer_config.json:   0%|          | 0.00/20.4k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/238k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.4k [00:00<?, ?B/s]

Accuracy: 0.7637
F1 Score: 0.8028


## 7. Fold Classification (Inference) with DeepProtT5

In [48]:
class T5ClassificationModel(PreTrainedModel):
    config_class = T5Config

    def __init__(self, config, d_model=None, num_classes=1195):
        super().__init__(config)
        self.num_classes = num_classes

        self.encoder = T5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_uniref50")

        hidden_dim = d_model if d_model is not None else config.d_model
        self.classification_head = nn.Linear(hidden_dim, num_classes)

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        labels=None,
        **kwargs
    ):
        encoder_outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        hidden_states = encoder_outputs.last_hidden_state

        mask = attention_mask.unsqueeze(-1)
        pooled_output = (hidden_states * mask).sum(dim=1) / mask.sum(dim=1)
        logits = self.classification_head(pooled_output)  # [batch_size, num_classes]

        loss = None
        if labels is not None:
            labels = labels.to(torch.bfloat16)
            loss = nn.CrossEntropyLoss()(logits, labels)

        return {
            "loss": loss,
            "logits": logits
        }

In [54]:
import torch
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score, accuracy_score

test = Fold(path + '/DeepProtein/data', 'test_superfamily_holdout')
test_protein_processed, test_target, test_protein_idx = collate_fn(test)


from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("jiaxie/DeepProtT5-Fold", do_lower_case=False, force_download=True)
model = T5ClassificationModel.from_pretrained("jiaxie/DeepProtT5-Fold", torch_dtype=torch.bfloat16).to("cuda")

model.eval()

predictions = []
true_labels = []

with torch.no_grad():
    for seq, target in zip(test_protein_processed, test_target):
        seq = " ".join(seq)
        encoded_input = tokenizer(seq, return_tensors="pt").to("cuda")
        outputs = model(**encoded_input)

        logits = outputs["logits"].cpu().to(torch.float32).squeeze()

        if logits.ndim == 0:
            pred = float(logits.numpy())
        else:
            pred = logits.numpy()

        predictions.append(pred)
        true_labels.append(target.numpy()[0])

predictions = torch.tensor(predictions, dtype=torch.float32)

if predictions.ndim == 1:
    probs = torch.sigmoid(predictions).numpy()
    binary_preds = (probs >= 0.5).astype(int)
else:
    probs = torch.softmax(predictions, dim=1).numpy()
    binary_preds = probs.argmax(axis=1)

true_labels = torch.tensor(true_labels).numpy()

accuracy = accuracy_score(true_labels, binary_preds)
f1 = f1_score(true_labels, binary_preds, average="weighted")

if predictions.ndim == 1:
    roc_auc = roc_auc_score(true_labels, probs)
    pr_auc = average_precision_score(true_labels, probs)
    print(f"ROC-AUC Score: {roc_auc:.4f}")
    print(f"PR-AUC Score: {pr_auc:.4f}")

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")


('STLYEKLGGTTAVDLAVDKFYERVLQDDRIKHFFADVDMAKQRAHQKAFLTYAFGGTDKYDGRYMREAHKELVENHGLNGEHFDAVAEDLLATLKEMGVPEDLIAEVAAVAGAPAHKRDVLNQ', 0)


tokenizer_config.json:   0%|          | 0.00/20.4k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/238k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.4k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/778 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.42G [00:00<?, ?B/s]

Accuracy: 0.7384
F1 Score: 0.7361
